# Group: CopyPaste
#### Dataset used: Dataset_Hospital_Vists.csv, test.csv
#### Participants: Ligia, Gina, Raza

### Content Draft

1. Defining the problem
2. Understanding the data: Notes from Kaggle
3. Data Quality Checks in detail
4. Exploratory Data Analysis

        4.1 Target Variable Distribution 
        4.2 Relationships between variables
5. Data Cleaning and Preprocessing

        5.1 Updating Existing Features
        5.2 Adding new features
6. Train-Test Split
7. Model Selection
8. Model Training
9. Model Evaluation
10. Model Intepretation
11. Fine-Tuning and Optimization
        



### Problem
Many patients miss their scheduled appointments.

### Goal 
Develop a machine learning model that predicts if a patient will miss a future appointment.

# Still need to check

#### Create heatmap of correlation
#### How does the time affect the outcome?
#### Is appointment date > scheduled data?
#### Distribution of the target variable.

# Thoughts on pre processing:
#### Change the no show to show to make it more intuitive.

# Preprocessing

### Missing Data:

#### Age, community, social welfare + disease info has missing data
